# Lab8 Probabilistic-Language-Modeling

## Part 1:Ngram

### 1.1 导入新闻数据

In [2]:
from nltk import ngrams
from collections import Counter
import os
import pandas as pd
import re
import jieba
import numpy as np
import operator
import csv


os.chdir("E:/graduate/class/Statistical Case Studies/homework7")
df = pd.read_csv("alldata.csv",encoding = "gb18030")
df.head(5)

,abstract,date,detail,href,origin_url,source,time,title,period
0,主要研究领域产业创新系统、技术创新、竞争战略、创新战略、管理整合及业务流程再造、开放式创新与...,2018/4/22,李显君：清华大学汽车工程系，汽车产业系统工程学科主任，博士生导师。主要研究领域产业创新系统、...,http://finance.sina.com.cn/china/2018-04-22/do...,NaN,新浪综合,16:30:29,美国制裁中兴通讯 敲响了中国四大警钟,3
1,美国制裁中兴通讯可能对中国出口构成“主要风险”】瑞穗沈建光、周雪称，由于中国科技行业的迅速崛...,2018/4/19,【瑞穗证券：美国制裁\n可能对中国出口构成“主要风险”】瑞穗沈建光、周雪称，由于中国科技...,http://finance.sina.com.cn/7x24/2018-04-19/doc...,NaN,新浪财经,18:24:42,瑞穗证券：美国制裁中兴通讯可能对中国出口构成“主要风险”,2
2,美国商务部表示，由于中兴通讯违反了曾与美国政府达成的和解协议，7年内禁止美国企业向中兴通讯出...,2018/4/20,当地时间16日，一记重拳向\n砸下。美国商务部表示，由于中兴通讯违反了曾与美国政府达成的和解...,http://finance.sina.com.cn/chanjing/gsnews/201...,NaN,央视财经,20:09:26,中兴直斥美国制裁不公平 中兴要“兴”应出什么牌？,3
3,彭新 4月17日，美国商务部禁止美国公司向中国主要手机生产厂商中兴通讯销售任何电子技术或通讯...,2018/4/18,彭新\n4月17日，美国商务部禁止美国公司向中国主要手机生产厂商\n销售任何电子技术或通讯元...,http://finance.sina.com.cn/chanjing/gsnews/201...,http://www.jiemian.com/article/2066223.html,界面,20:43:52,美国凭什么制裁中兴？ 企业遭到制裁后损失有多大？,2
4,富瑞表示，昂纳科技在光学元件的业务上，将会直接受惠于美国制裁中兴通讯．该行指，中兴通讯供应短...,2018/4/18,昂纳科技（00877）近期回落至5元水平，现价上升8.3%，报5.48元；成交约563万股，...,http://finance.sina.com.cn/stock/hkstock/ggscy...,NaN,新浪港股,13:16:30,昂纳科技被富瑞指将受惠中兴被美国制裁 现涨8%,2


### 1.2 定义文本预处理函数

In [3]:
def get_text(data):
    """获取csv文件中的新闻内容"""
    text=data["detail"]
    text = text.dropna() 
    len(text)
    text=[t.encode('utf-8').decode("utf-8") for t in text] 
    return text

def get_stop_words(file='stopWord.txt'):
    """去除停用词函数"""
    file = open(file, 'rb').read().decode('utf8').split(',')
    file = [line.strip() for line in file]
    return set(file)


def rm_tokens(words): 
    """去除停用词"""
    words_list = list(words)
    stop_words = get_stop_words()
    for i in range(words_list.__len__())[::-1]:
        if words_list[i] in stop_words:                      # 去除停用词
            words_list.pop(i)
        elif words_list[i].isdigit():
            words_list.pop(i)
    return words_list



def rm_char(text):
    """
    去除标点符号、特殊符号等，句子开头标记为S（for start），结尾标记为E（for end），为方便ngram模型去除与开头结尾相邻的词频
    """
    text = re.sub('\x01', '', text)                        #全角的空白符
    text = re.sub('\u3000', '', text) 
    text = re.sub(r"[\)(↓%·▲ \s+】【]","", text.encode("utf-8").decode("utf-8")) 
    text = re.sub(r"[\n\da-z+（）《》↗><‘’“”.-]","",text,flags=re.I)
    text = re.sub('\n+', " ", text)
    text = re.sub('[，、：。！？?；]', " ", text)
    text = re.sub(' +', "E S", text)
    return text

def convert_doc_to_wordlist(str_doc, cut_all=False):
    sent_list = str_doc.split('\n')
    sent_list = map(rm_char, sent_list)                       # 去掉一些字符，例如\u3000
    word_2dlist = [rm_tokens(jieba.cut(part, cut_all=cut_all))
                   for part in sent_list]                     # jieba分词
    word_list = sum(word_2dlist, [])
    return word_list

### 1.3 划分不同时间段的新闻
- 美国商务部：
    - **2018年4月16日晚**，美国商务部发布公告称，美国政府在未来7年内禁止中兴通讯向美国企业购买敏感产品。

- 中华人民共和国商务部：
    - **2018年4月17日**，商务部新闻发言人回应指出，中方一贯要求中国企业在海外经营过程中，遵守东道国的法律政策，合法合规开展经营。中兴公司与数百家美国企业开展了广泛的贸易投资合作，为美国贡献了数以万计的就业岗位。希望美方依法依规，妥善处理，并为企业创造公正、公平、稳定的法律和政策环境。商务部将密切关注事态进展，随时准备采取必要措施，维护中国企业的合法权益。
    - **2018年4月19日**，针对中兴被美国“封杀”的问题，商务部新闻发言人高峰在新闻发布会上表示，中方密切关注进展，随时准备采取必要措施，维护中国企业合法权益。

- 中兴通讯：
    - **2018年4月20日**，中兴通讯发布关于美国商务部激活拒绝令的声明，称在相关调查尚未结束之前，美国商务部工业与安全局执意对公司施以最严厉的制裁，对中兴通讯极不公平，“不能接受！”
    - **2018年4月22日晚间**，中兴通讯公告，2016年4月以来，公司吸取过去在出口管制合规方面的教训，高度重视出口管制合规工作，把合规视为公司战略的基石和经营的前提及底线。美国商务部工业与安全局激活拒绝令，公司已经且正在采取措施以遵守该拒绝令。公司积极与相关方沟通以及寻求解决方案。

- 划分四个时间段：
    - 4月17日
    - 4月18-4月19
    - 4月20-4月23
    - 4月24至今

In [4]:
def get_period_i(data,clean_text,i): 
    """#获取不同时间段的新闻文本"""
    j=0
    period_i=[]
    for j in list(range(len(clean_text))):
        if data['period'][j] == i:
            period_i.extend(clean_text[j])
    return period_i


### 1.4 获取Ngram词频统计

In [17]:
def getNgramDict(word_seg_list,n=2):
    """
    获得不同语料库的Ngram词频统计
    """
    rm_space = [w for w in word_seg_list if w not in [""," "]]
    
    analyzer2 = ngrams(rm_space,n)
    Ngram_dict = Counter(analyzer2)#字典的键值是元组

    #删除包含开头结尾的二元词频统计
    key_list=[]
    for key in Ngram_dict.keys():
        if "E" in key or "S" in key:
            key_list.append(key)
    
    for key in key_list:    
        del Ngram_dict[key]

    return Ngram_dict


clean_text=[convert_doc_to_wordlist(line) for line in get_text(df)]
period_1 = get_period_i(df,clean_text,1)
period_4 = get_period_i(df,clean_text,4)
all_period = [w for doc in clean_text for w in doc]


period_1_dict = getNgramDict(period_1,n=2)
period_4_dict = getNgramDict(period_4,n=2)
all_period_dict = getNgramDict(all_period,n=2)

print_n=50
n =2
sortedNGrams = sorted(period_1_dict.items(), key = operator.itemgetter(1), reverse=True) #=True 降序排列
print("the top %d wordcount of %d gram model of period_1 is:\n" %(print_n,n),sortedNGrams[:print_n],"/n")
sortedNGrams = sorted(period_4_dict.items(), key = operator.itemgetter(1), reverse=True) #=True 降序排列
print("the top %d wordcount of %d gram model of period_4 is:\n" %(print_n,n),sortedNGrams[:print_n],"/n")
sortedNGrams = sorted(all_period_dict.items(), key = operator.itemgetter(1), reverse=True) #=True 降序排列
print("the top %d wordcount of %d gram model of all_period is:\n" %(print_n,n),sortedNGrams[:print_n],"/n")

the top 50 wordcount of 2 gram model of period_1 is:
 [(('美国', '企业'), 128), (('中', '美'), 87), (('中国', '企业'), 72), (('违反', '美国'), 57), (('禁运', '事件'), 56), (('出口', '管制'), 48), (('贸易', '摩擦'), 47), (('自主', '可控'), 46), (('名', '员工'), 46), (('财经', '记者'), 46), (('第一', '财经'), 46), (('记者', '表示'), 46), (('拒绝', '令'), 44), (('高级', '雇员'), 44), (('达成', '和解'), 42), (('集成电路', '产业'), 42), (('中国', '商务部'), 40), (('禁止', '美国'), 40), (('和解', '协议'), 40), (('激活', '拒绝'), 40), (('美国商务部', '公司'), 38), (('名', '高级'), 36), (('美国', '供应商'), 36), (('出口', '限制'), 34), (('中兴', '美国'), 34), (('基带', '芯片'), 34), (('解雇', '名'), 34), (('公司', '激活'), 34), (('可能', '产生'), 32), (('市场', '约'), 32), (('约', '市场份额'), 32), (('积极', '沟通'), 32), (('设备', '市场'), 32), (('国产', '芯片'), 32), (('中国电信', '设备'), 31), (('产生', '影响'), 30), (('评估', '事件'), 30), (('正在', '全面'), 30), (('全面', '评估'), 30), (('通信', '设备'), 30), (('公司', '可能'), 30), (('事件', '公司'), 30), (('公司', '正在'), 30), (('亿美元', '罚款'), 30), (('国家', '安全'), 30), (('沟通', '应对'), 28), (('特朗普', '政府'), 28),

### 1.5 基于Ngram模型的文本分类器

根据每个类别的语料库训练各自的语言模型，实质上就是每一个类别都有一个概率分布，当新来一个文本的时候，只要根据各自的语言模型，计算出每个语言模型下这篇文本的发生概率，文本在哪个模型的概率大，这篇文本就属于哪个类别了

#### 1.5.1 计算二元词频概率

In [13]:
#calculate the corresponding probability for each entry in the indonesian LM  
sum_indo=0
for (k,v) in period_1_dict.items():  
    sum_indo += v  
for (k,v) in period_1_dict.items():  
    period_1_dict[k] = v/sum_indo  
sum_indo=0
for (k,v) in period_4_dict.items():  
    sum_indo += v  
for (k,v) in period_4_dict.items():  
    period_4_dict[k] = v/sum_indo  

#### 1.5.2 平滑化

period1 period4 的语料库中未出现过，而总语料库中出现过的词，赋值1e-8

In [14]:
#add one smoothing  
period_1_dict_s = period_1_dict.copy()
period_4_dict_s = period_4_dict.copy()
for key in  all_period_dict:
    if key not in period_1_dict_s:
        period_1_dict_s[key]=0.00000001
    if key not in period_4_dict_s:
        period_4_dict_s[key]=0.00000001

#### 查看预测结果

使用period2 period3的新闻作为测试集

In [ ]:
def getPrediction(clean_text_period):
    predict_period = []
    for doc in clean_text_period:
        doc_dict = getNgramDict(doc,n=2)
        period_1_pro = 1  
        period_4_pro = 1  
        for word in doc_dict:  
                #check if the word is in the entry, if it is, calculate the probability  
            if word in all_period_dict:  
                period_1_pro = period_1_pro + np.log(period_1_dict_s[word])  #乘多了就变0了
                period_4_pro = period_4_pro + np.log(period_4_dict_s[word])
                    
        if period_1_pro > period_4_pro: 
            predict_period.append(1)
#            print("the article is from period_1")
        elif period_1_pro < period_4_pro: 
            predict_period.append(4)
#            print("the article is from period_4")   
        else:
            predict_period.append(0)
#            print("can't predict the article's period")   
    return predict_period



#获取时期1 和4的新闻
df2 = df[df['period'].values==2].copy()
df3 = df[df['period'].values==3].copy()
clean_text2=[convert_doc_to_wordlist(line) for line in get_text( df2)]
clean_text3=[convert_doc_to_wordlist(line) for line in get_text( df3)]    

#不同时间段的新闻数量
df["period"].value_counts()

#查看预测结果
def seePrediction(data,i):
    df_i = data[data['period'].values==i].copy()
    clean_text_i=[convert_doc_to_wordlist(line) for line in get_text(df_i)]
    p_i = getPrediction(clean_text_i)
    df_i["pre"] = p_i
    date_pre = []
    for j in df_i.index:
        date_pre.append((df_i.ix[j,"date"],df_i.ix[j,"pre"]))
    return Counter(date_pre)

In [19]:
print("the prediction of period_2:\n",seePrediction(df,2),"\n")
print("the prediction of period_2:\n",seePrediction(df,3),"\n")

C:\Users\situ\Anaconda2\envs\py3\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


the prediction of period_2:
 Counter({('2018/4/18', 1): 47, ('2018/4/19', 1): 30, ('2018/4/19', 4): 30, ('2018/4/18', 4): 17, ('2018/4/18', 0): 1}) 

the prediction of period_2:
 Counter({('2018/4/22', 4): 11, ('2018/4/23', 4): 7, ('2018/4/20', 4): 7, ('2018/4/20', 1): 4, ('2018/4/21', 4): 3, ('2018/4/23', 1): 2, ('2018/4/22', 1): 2, ('2018/4/21', 1): 1, ('2018/4/22', 0): 1}) 



## Part 2 : Word2Vec

### 2.1 提取所有新闻文本并分词

无需去除标点符号

In [21]:
from gensim.models import word2vec  
import logging  

#把所有新闻文档放一起，分好词
os.chdir("E:/graduate/class/Statistical Case Studies/homework7")
def cutWord(line):
    line = re.sub(' +', " ",line)
    line = re.sub('\n+', " ",line)
    line = re.sub('\x01', '',line)                        #全角的空白符
    line = re.sub('\u3000', '', line)   
    seg_list = jieba.cut(line, cut_all=False)  
    return " ".join(seg_list)

cut_text=[cutWord(line) for line in get_text(df)]
print(" ".join(cut_text)[:1000])

李显君 ： 清华大学 汽车 工程系 ， 汽车产业 系统工程 学科 主任 ， 博士生 导师 。 主要 研究 领域 产业 创新 系统 、 技术创新 、 竞争 战略 、 创新 战略 、 管理 整合 及 业务流程 再造 、 开放式 创新 与 经营 模式 。   当 美国 对 叙利亚 狂 轰乱 炸 并 追加 制裁 俄罗斯 之际 ， 很多 国人 ， 包括 一些 号称 著名 战略家 以为 它 对 中国 的 贸易战 会 偃旗息鼓 ， 至少 现阶段 不会 再起 风浪 。 然而 ， 出乎 太 多 善良 国人 意料 的 是 ， 北京 时间 4 月 16 日 ， 特朗普 政府 突然 对 我国 中兴通讯 公司 实施 制裁 ， 定点 打击 。   担忧 与 绝望 、 愤怒 与 慷慨 、 决心 与 斗志 ， 瞬间 蔓延 于线 上线 下 。   而 我 要说 的 ： 感谢 特朗普 及 美国政府 这一 制裁 ！ 因为 ：   它 让 中国 彻底 明白 了 美国 贸易战 的 意图 ， 否则 我们 还 陶醉 在 对 美 大豆 等 高明 的 “ 七寸 反击 ” 中 ；   它 让 中国 彻底 清醒 冷静 对待 自身 的 实力 ， 否则 我们 还 沉醉于 “ 经济 奇迹 ” 和 “ 新 四大发明 ” 而 不能自拔 。   它 让 中国 决心 突破 制约 产业 的 核心 瓶颈 ， 否则 我们 会 在 芯片 等 核心技术 的 依赖 中 逐渐 丧失 竞争 资本 。   具体 而言 ， 美国 制裁 中兴通讯 ， 敲响 了 制约 中国崛起 的 四座 丧钟 ！ ? ?   很多 智库 和 战略 研究者 ， 根本 没有 预测 到 这场 来时 凶猛 的 贸易战 ， 时至今日 仍 有人 天真 地 认为 特朗普 很快 就 会 回到 谈判桌上 。   关于 中 美 贸易战 的 原因 ， 战略 短视 更是 无以复加 ， 主流 观点 是 ： 特朗普 为了 中期 选举 和 商人 敲诈 的 本性 。 这种 观点 对 我国 制定 长远 且 有 针对 的 对策 危害 至极 ， 比如 认为 此次 贸易战 原因 是 为了 中期 选取 ， 便 导致 了 我们 的 反击 重点 就是 美国 的 大豆 ， 因为 豆农是 特朗普 重要 的 票 仓 来源 。 但是 美国 社会 根本 不是 由 农民 左右 的 ， 而是 华尔街 ， 是 金融寡头 ！ 因此 ，

### 2.2 写入txt文件

word2vec要求导入utf-8的文本文件作为语料库，最好能达到1G以上

In [22]:
#all_cut_text = " ".join(cut_text)
f2 =open("fenci_result.txt", 'a')  
f2.write(" ".join(cut_text))  
f2.close()  

### 2.3 训练word2vec模型

In [24]:
logging.basicConfig(format='%(asctime)s:%(levelname)s: %(message)s', level=logging.INFO)  
sentences =word2vec.Text8Corpus(u"fenci_result.txt")  # 加载语料  
#Initialize a model
model =word2vec.Word2Vec(sentences, size=200)  #训练skip-gram模型，默认window=5  
print(model)  

2018-05-14 00:29:14,625:INFO: collecting all words and their counts
2018-05-14 00:29:14,631:INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-14 00:29:15,245:INFO: collected 22726 word types from a corpus of 1016251 raw words and 102 sentences
2018-05-14 00:29:15,250:INFO: Loading a fresh vocabulary
2018-05-14 00:29:15,300:INFO: min_count=5 retains 9653 unique words (42% of original 22726, drops 13073)
2018-05-14 00:29:15,300:INFO: min_count=5 leaves 981868 word corpus (96% of original 1016251, drops 34383)
2018-05-14 00:29:15,366:INFO: deleting the raw counts dictionary of 22726 items
2018-05-14 00:29:15,367:INFO: sample=0.001 downsamples 25 most-common words
2018-05-14 00:29:15,371:INFO: downsampling leaves estimated 626242 word corpus (63.8% of prior 981868)
2018-05-14 00:29:15,422:INFO: estimated required memory for 9653 words and 200 dimensions: 20271300 bytes
2018-05-14 00:29:15,426:INFO: resetting layer weights
2018-05-14 00:29:15,689:INFO: training

Word2Vec(vocab=9653, size=200, alpha=0.025)


### 2.4 查看词向量

In [26]:
# numpy vector of a word
print(model.wv['中美'])
len(model.wv['中美'])

[ -6.29079044e-01  -4.84883726e-01  -5.75685203e-01   4.33477551e-01
   4.81799960e-01  -5.07514298e-01   2.71963864e-03  -2.32927591e-01
   8.69941175e-01  -9.04398382e-01   3.59901547e-01   6.64961562e-02
   2.20925510e-01   1.21856458e-01   8.74979973e-01   5.11522293e-01
   8.02101493e-01  -2.76886821e-01   4.02252018e-01   1.68653533e-01
   5.66319287e-01   2.50396818e-01   4.54001367e-01  -6.03304148e-01
   3.81084174e-01  -2.37570018e-01   9.36034262e-01   5.52148938e-01
   3.57077986e-01   3.73060614e-01  -1.01284134e+00   5.82265377e-01
   4.00560915e-01   5.05854130e-01   4.84963469e-02   2.16357708e-01
  -3.99398804e-01   8.94760668e-01   1.06859155e-01   1.11635756e+00
  -8.23819518e-01  -3.96804154e-01  -2.30362236e-01  -4.82189894e-01
  -3.77694726e-01  -6.95044249e-02   9.85528901e-02  -6.19355202e-01
   1.92789122e-01   3.82011384e-01  -4.68891025e-01  -7.50730574e-01
   4.42582369e-01   2.69690275e-01   1.01212576e-01   1.71006233e-01
  -5.47219574e-01   1.19261837e+00

200

### 2.5 语义相关性的探索发现

In [35]:
# 计算两个词的相似度/相关程度  
words = ["制裁","禁令"]
#words = ["中兴","华为"]
try:  
    y1 = model.similarity(words[0],words[1])  
except KeyError:  
    y1 = 0  
print("【%s】和【%s】的相似度为%f" %(words[0],words[1],y1))  
print("-----------------------------------------------------\n")  
#  
# 计算某个词的余弦相关词列表  
#word = "制裁"
word = "中兴"
y2 = model.most_similar(word, topn=20)  # 20个最相关的  
print("和【%s】最相关的词有：\n" %word)  
for item in y2:  
    print(item[0], item[1])  
print("-----------------------------------------------------\n")  

   
# 寻找不合群的词 
words = u"制裁 禁令 封杀 打击 美国"
y4 =model.doesnt_match(words.split())  
print(u"【%s】中不合群的词：" % words,y4)  
print("-----------------------------------------------------\n")  
   
# 保存模型，以便重用  
model.save(u"中兴事件.model")  

C:\Users\situ\Anaconda2\envs\py3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """
C:\Users\situ\Anaconda2\envs\py3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
C:\Users\situ\Anaconda2\envs\py3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
2018-05-14 00:33:36,801:INFO: saving Word2Vec object under 中兴事件.model, separately None
2018-05-14 00:33:36,811:INFO: not storing attribute vectors_norm
2018-05-14 00:33:36,812:INFO: not storing attribute cum_table


【制裁】和【禁令】的相似度为0.897615
-----------------------------------------------------

和【中兴】最相关的词有：

中兴通讯 0.8524785041809082
法巴 0.7526480555534363
美国政府 0.7278330326080322
此次 0.714938223361969
伊朗 0.7128933072090149
华为 0.7055028676986694
俄罗斯 0.6956233382225037
俄 0.6800441145896912
其 0.6674633026123047
违反 0.6668750047683716
本次 0.6621092557907104
中兴公司 0.6578356027603149
禁运 0.6504078507423401
暂缓 0.6499906778335571
内容 0.6447697877883911
履行 0.6447207927703857
减轻 0.639124870300293
罚金 0.637557864189148
认定 0.6357647776603699
禁售 0.6334564685821533
-----------------------------------------------------

【制裁 禁令 封杀 打击 美国】中不合群的词： 美国
-----------------------------------------------------



2018-05-14 00:33:37,248:INFO: saved 中兴事件.model
